# Agents

AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages.
All agents share the following attributes and methods:

- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.name`: The unique name of the agent.
- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.description`: The description of the agent in text.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`: Send the agent a sequence of {py:class}`~autogen_agentchat.messages.ChatMessage` get a {py:class}`~autogen_agentchat.base.Response`. **It is important to note that agents are expected to be stateful and this method is expected to be called with new messages, not the complete history**.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`: Same as {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages` but returns an iterator of {py:class}`~autogen_agentchat.messages.AgentEvent` or {py:class}`~autogen_agentchat.messages.ChatMessage` followed by a {py:class}`~autogen_agentchat.base.Response` as the last item.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_reset`: Reset the agent to its initial state.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run` and {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run_stream`: convenience methods that call {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages` and {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` respectively but offer the same interface as [Teams](./teams.ipynb).

See {py:mod}`autogen_agentchat.messages` for more information on AgentChat message types.


## Assistant Agent

{py:class}`~autogen_agentchat.agents.AssistantAgent` is a built-in agent that
uses a language model and has the ability to use tools.

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)


## Getting Responses

We can use the {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages` method to get the agent response to a given message.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15), content=[FunctionCall(id='call_hqVC7UJUPhKaiJwgVKkg66ak', arguments='{"query":"AutoGen"}', name='web_search')]), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_hqVC7UJUPhKaiJwgVKkg66ak')])]
source='assistant' models_usage=RequestUsage(prompt_tokens=92, completion_tokens=14) content='AutoGen is a programming framework designed for building multi-agent applications.'


The call to the {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages` method
returns a {py:class}`~autogen_agentchat.base.Response`
that contains the agent's final response in the {py:attr}`~autogen_agentchat.base.Response.chat_message` attribute,
as well as a list of inner messages in the {py:attr}`~autogen_agentchat.base.Response.inner_messages` attribute,
which stores the agent's "thought process" that led to the final response.

```{note}
It is important to note that {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages`
will update the internal state of the agent -- it will add the messages to the agent's
history. So you should call this method with new messages.
**You should not repeatedly call this method with the same messages or the complete history.**
```

```{note}
Unlike in v0.2 AgentChat, the tools are executed by the same agent directly within
the same call to {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages`.
By default, the agent will return the result of the tool call as the final response.
```

You can also call the {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run` method, which is a convenience method that calls {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`. 
It follows the same interface as [Teams](./teams.ipynb) and returns a {py:class}`~autogen_agentchat.base.TaskResult` object.

## Streaming Messages

We can also stream each message as it is generated by the agent by using the
{py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` method,
and use {py:class}`~autogen_agentchat.ui.Console` to print the messages
as they appear to the console.

In [ ]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

---------- assistant ----------
[FunctionCall(id='call_fSp5iTGVm2FKw5NIvfECSqNd', arguments='{"query":"AutoGen information"}', name='web_search')]
[Prompt tokens: 61, Completion tokens: 16]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_fSp5iTGVm2FKw5NIvfECSqNd')]
---------- assistant ----------
AutoGen is a programming framework designed for building multi-agent applications. If you need more detailed information or specific aspects about AutoGen, feel free to ask!
[Prompt tokens: 93, Completion tokens: 32]
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 154
Total completion tokens: 48
Duration: 4.30 seconds


The {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` method
returns an asynchronous generator that yields each inner message generated by the agent,
with the final item being the response message in the {py:attr}`~autogen_agentchat.base.Response.chat_message` attribute.

From the messages, you can observe that the assistant agent utilized the `web_search` tool to
gather information and responded based on the search results.

You can also use {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run_stream` to get the same streaming behavior as {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`. It follows the same interface as [Teams](./teams.ipynb).

## Using Tools

Large Language Models (LLMs) are typically limited to generating text or code responses. 
However, many complex tasks benefit from the ability to use external tools that perform specific actions,
such as fetching data from APIs or databases.

To address this limitation, modern LLMs can now accept a list of available tool schemas 
(descriptions of tools and their arguments) and generate a tool call message. 
This capability is known as **Tool Calling** or **Function Calling** and 
is becoming a popular pattern in building intelligent agent-based applications.
Refer to the documentation from [OpenAI](https://platform.openai.com/docs/guides/function-calling) 
and [Anthropic](https://docs.anthropic.com/en/docs/build-with-claude/tool-use) for more information about tool calling in LLMs.

In AgentChat, the {py:class}`~autogen_agentchat.agents.AssistantAgent` can use tools to perform specific actions.
The `web_search` tool is one such tool that allows the assistant agent to search the web for information.
A custom tool can be a Python function or a subclass of the {py:class}`~autogen_core.tools.BaseTool`.

By default, when {py:class}`~autogen_agentchat.agents.AssistantAgent` executes a tool,
it will return the tool's output as a string in {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage` in its response.
If your tool does not return a well-formed string in natural language, you
can add a reflection step to have the model summarize the tool's output,
by setting the `reflect_on_tool_use=True` parameter in the {py:class}`~autogen_agentchat.agents.AssistantAgent` constructor.

### Langchain Tools

In addition to custom tools, you can also use tools from the Langchain library
by wrapping them in {py:class}`~autogen_ext.tools.langchain.LangChainToolAdapter`.

In [5]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
model_client = OpenAIChatCompletionClient(model="gpt-4o")
agent = AssistantAgent(
    "assistant", tools=[tool], model_client=model_client, system_message="Use the `df` variable to access the dataset."
)
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers?", source="user")], CancellationToken()
    )
)

---------- assistant ----------
[FunctionCall(id='call_BEYRkf53nBS1G2uG60wHP0zf', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')]
[Prompt tokens: 111, Completion tokens: 22]
---------- assistant ----------
[FunctionExecutionResult(content='29.69911764705882', call_id='call_BEYRkf53nBS1G2uG60wHP0zf')]
---------- assistant ----------
29.69911764705882
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 111
Total completion tokens: 22
Duration: 0.62 seconds


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='29.69911764705882', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=111, completion_tokens=22), content=[FunctionCall(id='call_BEYRkf53nBS1G2uG60wHP0zf', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='29.69911764705882', call_id='call_BEYRkf53nBS1G2uG60wHP0zf')], type='ToolCallExecutionEvent')])

## Using Model Context

{py:class}`~autogen_agentchat.agents.AssistantAgent` has a `model_context`
parameter that can be used to pass in a {py:class}`~autogen_core.model_context.ChatCompletionContext`
object. This allows the agent to use different model contexts, such as
{py:class}`~autogen_core.model_context.BufferedChatCompletionContext` to
limit the context sent to the model.

By default, {py:class}`~autogen_agentchat.agents.AssistantAgent` uses
the {py:class}`~autogen_core.model_context.UnboundedChatCompletionContext`
which sends the full conversation history to the model. To limit the context
to the last `n` messages, you can use the {py:class}`~autogen_core.model_context.BufferedChatCompletionContext`.

In [ ]:
from autogen_core.model_context import BufferedChatCompletionContext

# Create an agent that uses only the last 5 messages in the context to generate responses.
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
    model_context=BufferedChatCompletionContext(buffer_size=5),  # Only use the last 5 messages in the context.
)

## Other Preset Agents

The following preset agents are available:

- {py:class}`~autogen_agentchat.agents.UserProxyAgent`: An agent that takes user input returns it as responses.
- {py:class}`~autogen_agentchat.agents.CodeExecutorAgent`: An agent that can execute code.
- {py:class}`~autogen_ext.agents.openai.OpenAIAssistantAgent`: An agent that is backed by an OpenAI Assistant, with ability to use custom tools.
- {py:class}`~autogen_ext.agents.web_surfer.MultimodalWebSurfer`: A multi-modal agent that can search the web and visit web pages for information.
- {py:class}`~autogen_ext.agents.file_surfer.FileSurfer`: An agent that can search and browse local files for information.
- {py:class}`~autogen_ext.agents.video_surfer.VideoSurfer`: An agent that can watch videos for information.

## Next Step

Having explored the usage of the {py:class}`~autogen_agentchat.agents.AssistantAgent`, we can now proceed to the next section to learn about the teams feature in AgentChat.


<!-- ## CodingAssistantAgent

Generates responses (text and code) using an LLM upon receipt of a message. It takes a `system_message` argument that defines or sets the tone for how the agent's LLM should respond. 

```python

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)
`

We can explore or test the behavior of the agent by sending a message to it using the  {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`  method. 

```python
result = await writing_assistant_agent.on_messages(
    messages=[
        TextMessage(content="What is the weather right now in France?", source="user"),
    ],
    cancellation_token=CancellationToken(),
)
print(result) -->